In [ ]:
cd ..

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
# GLOABAL VARIABLES
completion_tokens = 0
prompt_tokens = 0

In [ ]:
import datetime
import json
import random
import openai as oai
from openai import OpenAI
from copy import copy, deepcopy

def get_final_answer(solution):
    # Finds final answer from string using //boxed highlight in LaTeX.
    start = solution.rfind("boxed") + 6
    end = start + 1
    bracket_counter = 0

    # Find the end of //boxed{} area
    while end < len(solution) and (solution[end] != '}' or bracket_counter != 0):
        if solution[end] == '{':
            bracket_counter += 1
        if solution[end] == '}':
            bracket_counter -= 1
        end += 1
        # Fail-safe for misused boxed at the end
    if end == len(solution):
        return "ERROR"
    return solution[start:end]

def gpt_usage(completion_tokens, prompt_tokens, backend):
    if backend == "gpt-4":
        cost = completion_tokens / 1000 * 0.06 + prompt_tokens / 1000 * 0.03
    elif backend == "gpt-3.5-turbo":
        cost = completion_tokens / 1000 * 0.002 + prompt_tokens / 1000 * 0.0015
    return {"completion_tokens": completion_tokens, "prompt_tokens": prompt_tokens, "cost": cost}


In [ ]:
from IPython.display import display, Math, Latex

def show_t(out):
    display(Latex(r"############################################"))
    display(Latex(r"$\textbf{Question " +str(out['record'].id) + ": }$"))
    display(Latex(r"$\textbf{Question: }$"))
    display(Latex(out['record'].question))
    display(Latex(r"$\textbf{Solution: }$"))
    display(Latex(out['record'].formatted_solution))
    display(Latex(r"$\textbf{Error type: }$" + out['error_type']))
    display(Latex(r"############################################"))
    display(Latex(r"$\textbf{GPT Solution: }$"))
    display(Latex((r"$\textbf{ERROR EXPLANATION: }$"  + out['error_explanation'])))
    display(Latex((r"$\textbf{STUDENT'S SOLUTION: }$"  +out['incorrect_solution'])))

In [ ]:
STEP1 = """
QUESTION: Tom has a red marble, a green marble, a blue marble, and three identical yellow marbles. How many different groups of two marbles can Tom choose?
CORRECT SOLUTION: There are two cases here: either Tom chooses two yellow marbles (1 result), or he chooses two marbles of different colors ($\\binom{4}{2}=6$ results). The total number of distinct pairs of marbles Tom can choose is $1+6=\\boxed{7}$.
ERROR TYPE: LOGICAL
ERROR EXPLANATION: Yellow marbles are thought as unique by the student. The student should have considered the yellow marbles as identical.

QUESTION: Simplify $\\sqrt[3]{1+8} \\cdot \\sqrt[3]{1+\\sqrt[3]{8}}$.
CORRECT SOLUTION: The first cube root becomes $\\sqrt[3]{9}$.\n$\\sqrt[3]{8}=2$, so the second cube root becomes $\\sqrt[3]{3}$.\nMultiplying these gives $\\sqrt[3]{27} = \\boxed{3}$.
ERROR TYPE: CONCENTRATION
ERROR EXPLANATION: Student took the square root instead of cube root in the first part of the expression which leads to an incorrect result for the entire expression.

QUESTION: How many total days were there in the years 2001 through 2004?
CORRECT SOLUTION: Including 2001 and 2004, there are 4 years total.\nHowever, 2004 is a leap year.\nTherefore, the number of days is $4\\times 365 + 1 = \\boxed{1461}$.
ERROR TYPE: CONCENTRATION
ERROR EXPLANATION: Student did not consider the leap year.

QUESTION: Find the greatest common divisor of 91 and 72.
CORRECT SOLUTION: $91 = 7^1 \\cdot 13^1$ and $72 = 2^3 \\cdot 3^2$, so gcd(91, 72) = $\\boxed{1}$.
ERROR TYPE: LOGICAL
ERROR EXPLANATION: Student performs the factorization correctly but fails to select greatest common divisor correctly.

Write an explanation for a potential error that could be performed while attempting to solve this question. If error type is LOGICAL, make sure that the incorrect result comes from a missing knowledge or misunderstanding in a subject. If the error type is CONCENTRATION, make sure that the incorrect result comes from a computation error or overlooking a variable etc. Explain the error in one-two sentences.

"""

In [ ]:
STEP2 = """
QUESTION: Tom has a red marble, a green marble, a blue marble, and three identical yellow marbles. How many different groups of two marbles can Tom choose?
CORRECT SOLUTION: There are two cases here: either Tom chooses two yellow marbles (1 result), or he chooses two marbles of different colors ($\\binom{4}{2}=6$ results). The total number of distinct pairs of marbles Tom can choose is $1+6=\\boxed{7}$.
ERROR TYPE: LOGICAL
ERROR EXPLANATION: Yellow marbles are thought as unique by the student. The student should have considered the yellow marbles as identical.
STUDENT'S SOLUTION: Tom can choose $ c\\left(\\begin{array}{l} 6 \\ 2 \\end{array}\\right)=\\frac{6.5}{2}=\\boxed{15} $.

QUESTION: Simplify $\\sqrt[3]{1+8} \\cdot \\sqrt[3]{1+\\sqrt[3]{8}}$.
CORRECT SOLUTION: The first cube root becomes $\\sqrt[3]{9}$.\n$\\sqrt[3]{8}=2$, so the second cube root becomes $\\sqrt[3]{3}$.\nMultiplying these gives $\\sqrt[3]{27} = \\boxed{3}$.
ERROR TYPE: CONCENTRATION
ERROR EXPLANATION: Student incorrectly simplified the expression which leads to an incorrect result for the entire expression.
STUDENT'S SOLUTION: The first cube root becomes $\\sqrt[3]{9}$.\n$\\sqrt[3]{8}=2$, so the second cube root becomes $\\sqrt[3]{2}$.\nMultiplying these gives $\\sqrt[3]{18} = \x08oxed{\\sqrt[3]{18}}$.

QUESTION: How many total days were there in the years 2001 through 2004?
CORRECT SOLUTION: Including 2001 and 2004, there are 4 years total.\nHowever, 2004 is a leap year.\nTherefore, the number of days is $4\\times 365 + 1 = \\boxed{1461}$.
ERROR TYPE: CONCENTRATION
ERROR EXPLANATION: Leap year is overlooked.
STUDENT'S SOLUTION: A year is 365 days. There are 4 years from 2001 through 2004.$4\\times 365 = \\boxed{1460}$.

QUESTION: Find the greatest common divisor of 91 and 72.
CORRECT SOLUTION: $91 = 7^1 \\cdot 13^1$ and $72 = 2^3 \\cdot 3^2$, so gcd(91, 72) = $\\boxed{1}$.
ERROR TYPE: LOGICAL
ERROR EXPLANATION: Student performs the factorization correctly but fails to select greatest common divisor.
STUDENT'S SOLUTION: $91 = 7 \\cdot 13$, $72 = 8 \\cdot 9$, so gcd(91, 72) = $\\boxed{13}$.

Act as a high school student attempting to solve the question, but made an error in the solution, which is characterized by ERROR TYPE and ERROR EXPLANATION.
Write down your erroneous solution steps.

"""

In [ ]:
from src.model.GPT import GPTModel
from src.datasets.MATH import MATH, MATHRecord

random.seed(42)

# Prepare dataset
dataset = MATH('data/math.jsonl')
dataset.records = [rec for rec in dataset.records if len(rec.formatted_solution) < 750]
# random.shuffle(dataset.records)
print(dataset.categories)

# Prepare model
gpt3 = GPTModel(model="gpt-3.5-turbo-0125", temperature=1, max_tokens=500)
# gpt4 = GPTModel(model="gpt-4-turbo-2024-04-09", temperature=1, max_tokens=500)

output_format = {
    'record': None,
    'error_type': None,
    'error_explanation': None,
    'incorrect_solution': None
}

In [ ]:
def run_step1(record, error):
    prompt = STEP1
    
    prompt += 'QUESTION: ' + record.question + '\n'
    prompt += 'CORRECT SOLUTION: ' + record.formatted_solution + '\n'
    prompt += 'ERROR TYPE: ' + error + '\n'
    prompt += 'ERROR EXPLANATION: '

    explanations = gpt3.query(prompt)
    return explanations[0]


def run_step2(record, error, explanation):

    prompt = STEP2
    
    prompt += 'QUESTION: ' + record.question + '\n'
    prompt += 'CORRECT SOLUTION: ' + record.formatted_solution + '\n'
    prompt += 'ERROR TYPE: ' + error + '\n'
    prompt += 'ERROR EXPLANATION: ' + explanation + '\n'
    prompt += 'STUDENT\'S SOLUTION: '

    solutions = gpt3.query(prompt)
    return solutions[0]


def run_for_category(cat, level, sample=5):
    outputs = []
    d = dataset.filter(cat, level)
    for i in range(sample):
        record = d[i]
        error = random.choice(['CONCENTRATION', 'LOGICAL'])
        out = copy(output_format)
        
        explanation = run_step1(record, error)
        solution = run_step2(record, error, 'exp')

        out['record'] = record
        out['error_type'] = error
        out['error_explanation'] = explanation
        out['incorrect_solution'] = solution

        outputs.append(out)
        
    return outputs

In [ ]:
# generate dataset for 140 questions
all_outputs = {}
for cat in dataset.categories:
    all_outputs[cat] = {}
    for lev in range(2, 6):
        all_outputs[cat][lev] = run_for_category(cat, lev, sample=5)

In [ ]:
print(gpt3.estimated_cost())

In [ ]:
import os
import pickle
path = 'output/080524_Experiment_TwoStepErrorDataset/'
os.makedirs(path, exist_ok=True)

In [ ]:
for cat in all_outputs.keys():
    for lev in all_outputs[cat].keys():
        filename = f"{cat.lower().replace(' ', '_')}_Level{lev}_Start0_End5.pkl"
        fp = path + filename
        pickle.dump(all_outputs[cat][lev], open(fp, 'wb'))

In [ ]:
outlist = []
for cat in all_outputs.keys():
    for lev in all_outputs[cat].keys():
        outlist += all_outputs[cat][lev]
pickle.dump(outlist, open(path+'all_questions.pkl', 'wb'))

---

## Notes
- Step 1 performed with GPT-3 and step 2 performed with GPT-4 but the performance did not increase
- GPT-4 costs 0.07629 per 5 questions, tends to generate much more tokens
- GPT-3 costs 0.00424 per 5 questions. For 100 questions, using gpt-3 in both steps, expected cost is 0.084$, ~9 cents.
- Even though error_explanation generated reasonably, GPT fails to convert it into a solution with error. GPT-4 also fails to do.
- GPT can not fail. Interesting finding.
- 140 questions costed 0.1532485 cents with 2 step approach